In [ ]:
import json
import time
import datetime
import random
import os


In [ ]:
import pandas as pd
import plotly.express as px

# Assuming df is your DataFrame containing the game data
import dash
from dash import dcc
from dash import html
import plotly.graph_objs as go

# Assuming app is your Dash application instance

# Define layout of the dashboard
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df = pd.read_json("./datadump/games_rm_1v1_s6.json")

print(df)

In [ ]:
main_df = df

In [ ]:

df = df.explode("teams")

print(df)

In [ ]:
from pandas import json_normalize
test_df = main_df
test_df.reset_index(drop=True, inplace=True)
test_df[['team1', 'team2']] = pd.json_normalize(test_df['teams'])

# Drop the original 'players' column
test_df.drop(columns=['teams'], inplace=True)
print(test_df)

In [ ]:
sample = pd.json_normalize(test_df['team2'])
print(len(sample.columns))

In [ ]:
player1_df = test_df
player1_df[['team_1','1','2','3']] = pd.json_normalize(test_df['team1'])

player1_df[['team_2','4','5']] = pd.json_normalize(test_df['team2'])
# player1_df = pd.json_normalize(test_df['team1'])
# player1_df = pd.json_normalize(player1_df['0'])
player1_df.drop(columns=['team1','1','2','3'], inplace=True)
player1_df.drop(columns=['team2','4','5'], inplace=True)
# Rename the columns with a custom prefix
# player1_df.columns = [f'team1{col}' for col in player1_df.columns]
# player1_df = pd.json_normalize(player1_df['team1'])
print(player1_df)

In [ ]:
sample1 = pd.json_normalize(player1_df['team_1'])
print(isinstance(sample1, pd.DataFrame))
sample2 = pd.json_normalize(player1_df['team_2'])
print(isinstance(sample2, pd.DataFrame))
combined_df = pd.concat([sample1, sample2], ignore_index=True)
print(len(combined_df))

In [ ]:
print(combined_df)

In [ ]:
avg_mmr_by_civilization = combined_df.groupby('civilization')['mmr'].mean().reset_index()
print(avg_mmr_by_civilization)

In [ ]:
win_loss_counts = combined_df.groupby(['civilization', 'result']).size().unstack(fill_value=0)
print(win_loss_counts)

In [ ]:


win_loss_counts['win_loss_ratio'] = win_loss_counts['win'] / win_loss_counts['loss']

fig = go.Figure(go.Bar(
    y=win_loss_counts.index,
    x=win_loss_counts['win_loss_ratio'],
    orientation='h'
))

# Customize layout
fig.update_layout(
    title='Win/Loss Ratio by Civilization',
    xaxis_title='Win/Loss Ratio',
    yaxis_title='Civilization',
    yaxis={'categoryorder':'total ascending'}
)

# Show the plot
fig.show()

In [ ]:
contoller = combined_df[combined_df['input_type'] == 'controller']
players_per_civilization = contoller.groupby('civilization').size()
print(contoller)
total_players = players_per_civilization.sum()

# Calculate the percentage of players who used the mouse input type for each civilization
percentage_players_mouse = (players_per_civilization / total_players) * 100


In [ ]:
fig = go.Figure(go.Bar(
    x=percentage_players_mouse.index,
    y=percentage_players_mouse,
    marker=dict(color='blue')  # Customize the color of the bars
))

# Customize layout
fig.update_layout(
    title='Percentage of Players per Civilization who Used Mouse Input Type',
    xaxis_title='Civilization',
    yaxis_title='Percentage of Players',
)

# Show the plot
fig.show()

In [ ]:
player1_data = sample1.rename(columns={'profile_id': 'player_id_1', "result" : 'result_1', "civilization" : "civ_1", "civilization_randomized": "random_civ_1",
                                        "rating": "rating_1", "rating_diff" : "rate_diff_1", "mmr" : "mmr_1", "input_type" : "input_type_1"})
player2_data = sample2.rename(columns={'profile_id': 'player_id_2', "result" : 'result_2', "civilization" : "civ_2", "civilization_randomized": "random_civ_2",
                                        "rating": "rating_2", "rating_diff" : "rate_diff_2", "mmr" : "mmr_2", "input_type" : "input_type_2"})
teamdata = pd.concat([player1_data, player2_data], axis=1)
print(teamdata)

In [ ]:
frame  = teamdata
frame = frame[(frame['civ_1'] != 'unknown') & (frame['civ_2'] != 'unknown') & frame['result_1'].isin(['win', 'loss']) & frame['result_2'].isin(['win', 'loss']) ]
frame['p1_wins'] = frame['result_1'] == 'win'
frame['p2_wins'] = frame['result_2'] == 'win'
success_rate = frame.groupby(['civ_1', 'civ_2'])['p1_wins'].mean().unstack()
print(success_rate)


In [46]:
lol = frame.groupby(['civ_1', 'civ_2'])['p1_wins'].mean()
print(lol)

civ_1            civ_2              
abbasid_dynasty  abbasid_dynasty        0.467975
                 ayyubids               0.395062
                 byzantines             0.477362
                 chinese                0.464075
                 delhi_sultanate        0.453099
                                          ...   
zhu_xis_legacy   mongols                0.480408
                 order_of_the_dragon    0.510728
                 ottomans               0.490442
                 rus                    0.495631
                 zhu_xis_legacy         0.480876
Name: p1_wins, Length: 256, dtype: float64


In [47]:
lol2 = frame.groupby(['civ_2', 'civ_1'])['p2_wins'].mean()
print(lol2)

civ_2            civ_1              
abbasid_dynasty  abbasid_dynasty        0.532025
                 ayyubids               0.471033
                 byzantines             0.565338
                 chinese                0.554261
                 delhi_sultanate        0.537143
                                          ...   
zhu_xis_legacy   mongols                0.539336
                 order_of_the_dragon    0.563978
                 ottomans               0.546029
                 rus                    0.569843
                 zhu_xis_legacy         0.519124
Name: p2_wins, Length: 256, dtype: float64


In [50]:
success_rate = (lol + lol2)/2
success_rate = success_rate.unstack()
print(success_rate)

civ_2                abbasid_dynasty  ayyubids  byzantines   chinese  \
civ_1                                                                  
abbasid_dynasty             0.500000  0.433047    0.521350  0.509168   
ayyubids                    0.566953  0.500000    0.606122  0.586019   
byzantines                  0.478650  0.393878    0.500000  0.512931   
chinese                     0.490832  0.413981    0.487069  0.500000   
delhi_sultanate             0.504879  0.458412    0.535489  0.523893   
english                     0.545120  0.473042    0.524695  0.532039   
french                      0.470307  0.400573    0.501799  0.527805   
holy_roman_empire           0.528417  0.414595    0.502354  0.549345   
japanese                    0.527215  0.419312    0.540602  0.581622   
jeanne_darc                 0.546557  0.466789    0.598029  0.583540   
malians                     0.463208  0.472194    0.480874  0.514454   
mongols                     0.505310  0.470425    0.544813  0.52

In [51]:
# Create a heatmap
fig = go.Figure(data=go.Heatmap(
    z=success_rate.values,
    x=success_rate.columns,
    y=success_rate.index,
    colorscale='Viridis',
    colorbar=dict(title='Success Rate'),
))

# Customize layout
fig.update_layout(
    title='Success Rate of p1 vs p2 by Civilization',
    xaxis_title='Civ_2',
    yaxis_title='Civ_1',
)

# Show the plot
fig.show()

In [ ]:
win_loss_counts['win_loss_ratio'] = win_loss_counts['win'] / win_loss_counts['loss']

fig = go.Figure(go.Bar(
    y=win_loss_counts.index,
    x=win_loss_counts['win_loss_ratio'],
    orientation='h'
))

# Customize layout
fig.update_layout(
    title='Win/Loss Ratio by Civilization',
    xaxis_title='Win/Loss Ratio',
    yaxis_title='Civilization',
    yaxis={'categoryorder':'total ascending'}
)

# Show the plot
fig.show()

In [ ]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Average MMR by Civilization"),
    dcc.Graph(
        id='civilization-mmr-bar-chart',
        figure={
            'data': [
                go.Bar(
                    x=avg_mmr_by_civilization['civilization'],
                    y=avg_mmr_by_civilization['mmr'],
                    marker=dict(color=avg_mmr_by_civilization['mmr']),
                    hoverinfo='y'
                )
            ],
            'layout': {
                'title': 'Average MMR by Civilization',
                'xaxis': {'title': 'Civilization'},
                'yaxis': {'title': 'Average MMR'}
            }
        }
    )
])

# Run the Dash application
# if __name__ == '__main__':


In [ ]:
app.run_server(debug=True)